In [22]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle

In [5]:
pd.__version__

'1.4.2'

In [6]:
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

 ### Question 1

In [7]:
df_jan. shape , df_feb.shape

((3066766, 19), (2913955, 19))

In [8]:
print(f"There are  {df_jan.shape[1]-1} Columns in this this dataset")

There are  18 Columns in this this dataset


## Question 2

In [9]:
df_jan["duration"] = (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]).dt.total_seconds()/60
df_jan["duration"].std()

42.594351241920904

## Question 3

In [10]:
df_jan_filtred = df_jan[(df_jan["duration"] >=1)&(df_jan["duration"] <=60)]

In [11]:
df_jan_filtred.shape, df_jan.shape

((3009173, 20), (3066766, 20))

In [12]:
fraction = len(df_jan_filtred)/len(df_jan)
print(f"The fraction is : {fraction:.0%}")

The fraction is : 98%


## Question  4

In [14]:
categorical = ['PULocationID', 'DOLocationID']
df_jan_filtred[categorical] = df_jan_filtred[categorical].astype(str)

/tmp/ipykernel_5861/1761852564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_filtred[categorical] = df_jan_filtred[categorical].astype(str)


In [17]:
dv = DictVectorizer()
train_dicts = df_jan_filtred[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [18]:
print('Q4:', X_train.shape[1])

Q4: 515


## Question 5

In [19]:
y_train = df_jan_filtred.duration.values
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
print('Q5:', round(rmse_train, 2))

Q5: 7.65


## Question 6

In [20]:
# Q6: Évaluation sur février
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds() / 60
df_feb_filtered = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
df_feb_filtered[categorical] = df_feb_filtered[categorical].astype(str)

val_dicts = df_feb_filtered[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb_filtered.duration.values
y_pred_val = model.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)
print('Q6:', round(rmse_val, 2))

/tmp/ipykernel_5861/4066412140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered[categorical] = df_feb_filtered[categorical].astype(str)


Q6: 7.81


In [24]:
import pickle

output_file = 'model_linreg_dv.pkl'

model_artifacts = {
    'model': model,
    'dict_vectorizer': dv
}

with open(output_file, 'wb') as f_out:
    pickle.dump(model_artifacts, f_out)

print(f"✅ Modèle sauvegardé dans {output_file}")



✅ Modèle sauvegardé dans model_linreg_dv.pkl


In [23]:

with open('model_linreg_dv.pkl', 'rb') as f_in:
    saved_objects = pickle.load(f_in)

model = saved_objects['model']
dv = saved_objects['dict_vectorizer']


FileNotFoundError: [Errno 2] No such file or directory: 'model_linreg_dv.pkl'